In [1]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

import time
import cyipopt as ipopt

from utils import build_ipopt

In [2]:
from model_utils import NNSolver_eq_proj

filepath = "/home/jxxiong/A-xjx/DC3/datasets/nonconvex/random_nonconvex_dataset_var100_ineq50_eq50_ex10000"
with open(filepath, 'rb') as f:
    data = pickle.load(f)

arg_path = "/home/jxxiong/A-xjx/DC3/results/NonconvexProblem-100-50-50-10000/method_eq_proj/2706e1212f81176f70decab0cefb4410cb753c77/1713357205-2286425/args.dict"
args = pickle.load(open(arg_path, 'rb'))

model = NNSolver_eq_proj(data, args)

model_path = "/home/jxxiong/A-xjx/DC3/results/NonconvexProblem-100-50-50-10000/method_eq_proj/2706e1212f81176f70decab0cefb4410cb753c77/1713357205-2286425/solver_net.dict"
model.load_state_dict(torch.load(model_path))

model.eval()
# model.to('cuda')
model.to('cpu')

NNSolver_eq_proj(
  (net): Sequential(
    (0): Linear(in_features=50, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=200, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU(approximate='none')
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=200, out_features=100, bias=True)
  )
)

In [3]:
DEVICE = 'cpu'
for attr in dir(data):
    var = getattr(data, attr)
    if not callable(var) and not attr.startswith("__") and torch.is_tensor(var):
        try:
            setattr(data, attr, var.to(DEVICE))
        except AttributeError:
            pass

In [4]:
A = data.A
G = data.G
h = data.h
Q = data.Q
p = data.p

A_np = data.A_np
G_np = data.G_np
h_np = data.h_np
Q_np = data.Q_np
p_np = data.p_np

X = data.X
X_np = data.X_np

num_var = A.shape[1]
num_eq = A.shape[0]
num_ineq = G.shape[0]

In [5]:
def get_init(A, x):
    y0 = np.linalg.pinv(A)@x
    return y0

In [6]:
idx = 9000

In [7]:
full_model = build_ipopt(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
start_time = time.time()
optimal_x_full, info_full = full_model.solve(get_init(A_np, X_np[idx]))
end_time = time.time()
time_full = end_time - start_time
optimal_val_full = info_full['obj_val']
active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
active_ineq_target.tolist()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



/home/jxxiong/A-xjx/DC3/utils.py:769: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


[1, 3, 4, 12, 13, 18, 34, 37, 39, 40, 42, 45, 46]

In [8]:
# calculate the parameter values

opt_gap = 0.01
t = num_ineq / opt_gap / abs(optimal_val_full)

dual_threshold = 1e-3
ineq_threshold = 1 / dual_threshold / int(t)

print(int(t), ineq_threshold)

431 2.320185614849188


# remove part of the inequality constraints

In [9]:
# XX = X[idx].unsqueeze(0).to('cuda')
# start_time = time.time()
# Yhat = model(XX)
# Ystar = data.projection(XX, Yhat)
# end_time = time.time()

# start_time - end_time


In [10]:
start_time = time.time()
Yhat = model(X[idx].unsqueeze(0))
Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
end_time = time.time()
inference_time = end_time - start_time
print("Inference time: ", inference_time)
Ystar = Ystar.detach().numpy().flatten()
ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
ineq_sorted = np.sort(-G_np@Ystar + h_np)
print(ineq_sorted)
print(ineq_sorted_index)
inactive_ineq_index = ineq_sorted_index[ineq_sorted > ineq_threshold]
# inactive_ineq_index = ineq_sorted_index[ineq_sorted > 3.36]
print(inactive_ineq_index)

Inference time:  0.0012979507446289062
[ 0.33618555  0.39220554  0.45107663  0.50017045  0.52421756  0.56971751
  0.58069428  0.59901621  0.63224894  0.63798819  0.88612242  0.94228262
  0.99003326  1.07726453  1.07857428  1.13645375  1.27553673  2.10891739
  2.13518904  2.17800478  2.19373177  2.22177968  2.68412888  2.7667148
  2.90988385  3.23770211  3.44838402  3.52140208  3.56906934  3.9138547
  4.09076069  4.98843254  5.25249889  5.58059262  5.75191399  6.15332183
  6.350956    6.47356779  7.48299012 10.37707413 10.37806946 10.57881741
 10.91353806 11.25204565 11.93099774 12.54175085 12.70290288 13.26941813
 14.7172558  14.97075727]
[ 3 13 18 34 37 39  4  1 45 12 42  9 40 46 20 29 38 19  5 16 32 24 11 31
 43 28  0  2 22 23  8 17 26 10 41 44 35 15 36  6 21 25 48 33  7 30 47 14
 49 27]
[11 31 43 28  0  2 22 23  8 17 26 10 41 44 35 15 36  6 21 25 48 33  7 30
 47 14 49 27]


In [11]:
set(inactive_ineq_index).intersection(set(active_ineq_target))

set()

In [12]:
np.setdiff1d(np.arange(num_ineq), active_ineq_target).shape

(37,)

In [13]:
active_ineq_target

array([ 1,  3,  4, 12, 13, 18, 34, 37, 39, 40, 42, 45, 46])

In [14]:
# remaining_ineq_index = np.setdiff1d(np.arange(num_ineq), inactive_ineq_index)
# remaining_ineq_index = active_ineq_target[60:]
remaining_ineq_index = np.array(active_ineq_target.tolist() + [0, 2, 11, 14, 15])


G_remaining = G_np[remaining_ineq_index]
h_remaining = h_np[remaining_ineq_index]
model_reduced = build_ipopt(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining)
start_time = time.time()
optimal_x_reduced, info_reduced = model_reduced.solve(get_init(A_np, X_np[idx]))
end_time = time.time()
optimal_val = info_reduced['obj_val']
time_reduced = end_time - start_time
# time_reduced = end_time - start_time + inference_time
print("Value: full={}, reduced={}, diff={}".format(optimal_val_full, optimal_val, optimal_val_full - optimal_val))
print("Time: full={}, reduced={}, diff={}".format(time_full, time_reduced, time_full - time_reduced))

Value: full=-11.596384930012848, reduced=-11.596384939487834, diff=9.47498612902109e-09
Time: full=0.042676448822021484, reduced=0.03588604927062988, diff=0.0067903995513916016


# osqp warm start

In [24]:
num_trial = 10

total_time = 0.0
for _ in range(num_trial):
    solver = build_ipopt(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    # solver1.warm_start(x=Ystar)
    start_time = time.time()
    x, results = solver.solve(get_init(A_np, X_np[idx]))
    end_time = time.time()
    total_time += (end_time - start_time)
print(total_time / num_trial)
print(results['obj_val'])


0.041019248962402347
-11.596384930012848


In [25]:
total_time_ws = 0.0
for _ in range(num_trial):
    solver_ws = build_ipopt(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    start_time = time.time()
    x_ws, result_ws = solver_ws.solve(Ystar)
    end_time = time.time()
    total_time_ws += (end_time - start_time)
print(total_time_ws / num_trial + inference_time)
print(result_ws['obj_val'])

/home/jxxiong/A-xjx/DC3/utils.py:769: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


0.03576536178588867
-11.596384959973047


In [26]:
total_time_reduced = 0.0
for _ in range(num_trial):
    solver_reduced = build_ipopt(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining)
    start_time = time.time()
    x_reduced, results_reduced = solver_reduced.solve(get_init(A_np, X_np[idx]))
    end_time = time.time()
    total_time_reduced = end_time - start_time
print(total_time_reduced / num_trial + inference_time)
print(results_reduced['obj_val'])

/home/jxxiong/A-xjx/DC3/utils.py:769: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


0.0042236089706420895
-11.596384939487834


In [20]:
total_time_reduced_ws = 0.0
for _ in range(num_trial):
    solver_reduced_ws = build_ipopt(Q_np, p_np, A_np, X_np[idx], G_remaining, h_remaining)
    start_time = time.time()
    x_reduced_ws, results_reduced_ws = solver_reduced_ws.solve(Ystar)
    end_time = time.time()
    total_time_reduced_ws = end_time - start_time
print(total_time_reduced_ws / num_trial + inference_time)
print(results_reduced_ws['obj_val'])
print(total_time_reduced_ws / num_trial)

0.001326131820678711
-15.492729794298866
2.8181076049804688e-05


# check

In [ ]:
error = []
for i in range(9300, 10000):
    full_model = build_gurobi_model(Q_np, p_np, A_np, X_np[idx], G_np, h_np)
    full_model.optimize()
    optimal_x_full = np.array([v.x for v in full_model.getVars()])
    active_ineq_target = np.where(np.abs(G_np@optimal_x_full - h_np) <= 1e-4)[0]
    active_ineq_target.tolist()

    Yhat = model(X[idx].unsqueeze(0))
    Ystar = data.projection(X[idx].unsqueeze(0), Yhat)
    Ystar = Ystar.detach().numpy().flatten()
    ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
    ineq_sorted = np.sort(-G_np@Ystar + h_np)
    inactive_ineq_index = ineq_sorted_index[ineq_sorted > 2]

    error.append(len(set(inactive_ineq_index).intersection(set(active_ineq_target))))
print(max(error))



0


In [ ]:
data.opt_solve(X[idx].unsqueeze(0))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



/home/jxxiong/A-xjx/DC3/utils.py:688: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(
/home/jxxiong/.conda/envs/scopf/lib/python3.9/site-packages/cyipopt/utils.py:43: FutureWarning: The function named 'addOption' will soon be deprecated in CyIpopt. Please replace all uses and use 'add_option' going forward.
  warnings.warn(msg, FutureWarning)


(array([[-0.55611482, -0.56368449, -0.64790394,  0.37884889, -0.19126514,
         -0.1764256 ,  0.71853515, -0.4212393 , -0.6760313 , -0.27583975,
         -0.45301068, -0.66199673, -0.47733795, -0.30222725, -1.00513773,
          0.35989204, -1.40671222,  0.0209178 , -0.34485383, -0.68199178,
          0.2479107 , -0.16290416,  0.54918279, -0.70889106,  0.28346117,
         -0.31472916,  0.67713666, -0.96370784, -0.09016452, -0.46824194,
         -0.67885591,  0.65289059, -0.55205763, -0.81256397,  0.35689147,
         -1.57463091,  0.43449316, -0.3146935 ,  0.31734281, -0.09843636,
         -0.04760909, -1.00478053, -0.13062429, -0.23469302, -0.49049767,
         -0.2889595 ,  0.00300261, -0.75304159, -0.48184272, -0.34045444,
         -0.12102274,  0.06690322, -0.4348534 , -0.34388626,  0.02230891,
         -0.12307867,  0.69059265, -0.76568063,  0.22664308,  0.587296  ,
         -0.5935183 , -0.17313994, -0.58612199, -0.89719827, -0.18061466,
         -0.85654165, -0.75405419, -0.